<img src="https://www.unir.net/wp-content/uploads/2019/11/Unir_2021_logo.svg" width="240" height="240" align="right"/>

<center><h1>Técnicas de Inteligencia Artificial</header1></center>
<left><h1>Actividad 1. Laboratorio: Árboles de decisión, reglas y ensemble learning</header1></left>

Presentado por: Carlos Felipe Pimienta Moreno  <br>
Fecha: 10/05/2024

## Explique el problema a resolver. 
Descripción del problema. Tipo de problema (justifique). Variable objetivo, variables de entrada. Utilidad de su posible solución. Elementos adicionales que considere relevantes:

## Importación de librerias necesarias

In [356]:
#Para esta actividad se importarán las siguientes librerías:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import sklearn as sl
plt.style.use('ggplot')
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.tree import plot_tree
from sklearn.tree import export_graphviz
from sklearn.tree import export_text
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import pydotplus
from IPython.display import Image as PImage
from subprocess import check_call
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from PIL import Image, ImageDraw, ImageFont
import os,sys
from scipy import stats
from sklearn.linear_model import LinearRegression

## Cargar el Dataset

In [370]:
#Código para cargar el Dataset
#url = 'https://raw.githubusercontent.com/oscar-unir/TIA/main/datasets/car/Laboratorio_dataset_car.csv'

df = pd.read_csv('https://raw.githubusercontent.com/Felipe-UNIR/TecnicasIA-datasets/main/Laboratorio_dataset_car.csv',sep=';')
#df.head(10)
df

,Buying,Maintenance,Doors,Person,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1745,low,low,5more,more,med,high,vgood
1746,low,low,5more,more,med,high,vgood
1747,low,low,5more,more,big,low,unacc
1748,low,low,5more,more,big,med,good


Número de instancias en total

In [383]:
df.shape[0]

1750

## Caracterización del Dataset

Se incluye una descripción de los datos con:

>- Número de clases de la variable objetivo, indicando que representan dichas clases y el tipo de valor que toman.
>- Número de instancias en total.
>- Número de instancias pertenecientes a cada clase.
>- Número de atributos de entrada, su significado y tipo.
>- ¿Hay algún valor de atributo desconocido?

Se incorporá una descripción (EDA) del conjunto de datos utilizado. Se analiza el dataset proporcionando, se muestra al menos algunas de sus características mediante tablas y al menos algunas de ellas en modo gráfico (p.ej., histogramas, diagramas de dispersión, diagramas de cajas y bigotes, etc.)


Diferentes clases, mediante el metodo unique

In [395]:
comprar = np.unique(df['Buying'])
print(f'Buying: {comprar}')

mantenimiento = np.unique(df['Maintenance'])
print(f'Maintenance: {mantenimiento}')

puertas = np.unique(df['Doors'])
print(f'Doors: {puertas}')

personas = np.unique(df['Person'])
print(f'Person: {personas}')

maletero = np.unique(df['lug_boot'])
print(f'lug_boot: {maletero}')

seguridad = np.unique(df['safety'])
print(f'safety: {seguridad}')

clases = np.unique(df['class'])
print(f'class: {clases}')

Buying: ['high' 'low' 'med' 'vhigh']
Maintenance: ['high' 'low' 'med' 'vhigh']
Doors: ['2' '3' '4' '5more']
Person: ['2' '4' 'more']
lug_boot: ['big' 'med' 'small']
safety: ['high' 'low' 'med']
class: ['acc' 'good' 'unacc' 'vgood']


Tipo de dato de la clase

In [406]:
np.array(df['class']).dtype

dtype('O')

Numero de muestras por clase

In [416]:
df['class'].value_counts()

class
unacc    1215
acc       390
good       75
vgood      70
Name: count, dtype: int64

Dividad entradas y clases, se considera solo la columna "class"

In [425]:
y = df['class']

Se considera las columnas restantes

In [426]:
X = df.drop(columns=['class'])

Entrenamiento de datos

In [427]:
X_df = df
y_df = X

Número de atributos de la entrada

In [428]:
X.describe()

,Buying,Maintenance,Doors,Person,lug_boot,safety
count,1750,1750,1750,1750,1750,1750
unique,4,4,4,3,3,3
top,vhigh,low,2,4,big,high
freq,443,447,444,587,585,590


En un par de párrafos haga un resumen de los principales hallazagos encontrados:    

## Preprocesamiento del dataset. Transformaciones previas necesarias para la modelación

In [474]:
#Código que realice las transformaciones necesarias para poder realizar los procesos de modelación. Ej.One hot enconding

## División del dataset en datos de entrenamiento y datos de test 

In [479]:
#Código que realice la división en entrenamiento y test, de acuerdo con la estretgia de evaluación planeada. Describa cuál es.
X_train, X_validation, y_train, y_validation = train_test_split(X_df, y_df, test_size = 0.2, random_state = 42)

Arbol de decisión CART seleccionando atributos GINI <br>
Entrenamiento

In [480]:
arbol_gini = DecisionTreeClassifier(criterion='gini')

In [481]:
y_validation

,Buying,Maintenance,Doors,Person,lug_boot,safety
1246,med,low,3,4,med,high
862,high,low,5more,4,med,high
298,vhigh,med,4,more,med,low
855,high,low,5more,2,big,med
1534,low,med,2,2,med,high
...,...,...,...,...,...,...
1146,med,med,4,2,small,med
704,high,med,3,more,small,low
438,vhigh,low,5more,more,med,med
715,high,med,4,2,small,high


In [482]:
arbol_gini.fit(X_train,y_train)
predictions_gini = arbol_gini.predict(X_validation)

ValueError: could not convert string to float: 'vhigh'

Se evalua las predicciones, se evalua la predicción obtenida

In [ ]:
print(accuracy_score(y_validation, predictions_gini))

NameError: name 'predictions_gini' is not defined

## Ajuste de los modelos de clasificación propuestos

Justifique la selección de las dos propuestas de modelación seleccionadas:

In [ ]:
#Código de ajuste del modelo de clasificación 1

In [ ]:
#Código de ajuste del modelo de clasificación 2

## Evaluación de cada modelo

Al menos incluya:

+ Instancias clasificadas correctamente
+ Instancias clasificadas incorrectamente
+ TP Rate
+ FP Rate
+ Matriz de confusión


In [ ]:
#Código para mostrar la evaluación del modelo de clasificación 1

Construya un párrafo con los principales hallazgos.

In [ ]:
#Código  para mostrarla evaluación del modelo de clasificación 2

Construya un párrafo con los principales hallazgos.

## Comparación del desempeño de modelos

In [ ]:
#Código para mostrar la comparación de métricas de desempeño de las dos propuestas en tabla

In [ ]:
#Código para mostrar la comparación de métricas de desempeño de las dos propuestas en gráfica

Construya un párrafo con los principales hallazgos.

## Discusión de los resultados obtenidos y argumentos sobre cómo se podrían mejorar de dichos resultados

Realice en este espacio todo el análsis de resultados final incluyendo: ventajas y desventajas de cada modelo propuesto, Resultados comparados. Conclusiones objetivas y significantes con base a las diferentes métricas escogidas. Recomendaciones de mejora de las propuestas: 